In [1]:
module O

"""
Implement `elzero(x)`, `Base.:+(x, y)`, `Base.length(x)`, and `Base.getindex(x, i)` for a subtype `T` of `AbstractFoo`.  Then you can use `naivesum(x)` method for `x::T`.
"""
abstract type AbstractFoo end
function naivesum(x::AbstractFoo)
    s = elzero(x)
    for i in 1:length(x)
        s += x[i]
    end
    s
end

struct Foo <: AbstractFoo
    a
end
elzero(x::Foo) = Foo(zero(eltype(x.a)))
Base.:+(x::Foo, y::Foo) = Foo(x.a + y.a)
Base.length(x::Foo)::Int = length(x.a)
Base.getindex(x::Foo, i::Integer) = Foo(x.a[i])

struct Bar{T} <: AbstractFoo
    a::T
end
elzero(x::Bar) = Bar(zero(eltype(x.a)))
Base.:+(x::Bar, y::Bar) = Bar(x.a + y.a)
Base.length(x::Bar) = length(x.a)
Base.getindex(x::Bar, i::Integer) = Bar(x.a[i])

end

@doc O.AbstractFoo

Implement `elzero(x)`, `Base.:+(x, y)`, `Base.length(x)`, and `Base.getindex(x, i)` for a subtype `T` of `AbstractFoo`.  Then you can use `naivesum(x)` method for `x::T`.


In [2]:
a = randn(10^6)
foo = O.Foo(a)
bar = O.Bar(a);

In [3]:
@code_warntype O.naivesum(foo)

MethodInstance for Main.O.naivesum(::Main.O.Foo)
  from naivesum(x::Main.O.AbstractFoo) in Main.O at In[1]:7
Arguments
  #self#::Core.Const(Main.O.naivesum)
  x::Main.O.Foo
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  s::Main.O.Foo
  i::Int64
Body::Main.O.Foo
1 ─       (s = Main.O.elzero(x))
│   %2  = Main.O.length(x)::Int64
│   %3  = (1:%2)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Main.O.Foo
│   %12 = Base.getindex(x, i)::Main.O.Foo
│         (s = %11 + %12)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s



In [4]:
@code_warntype O.naivesum(bar)

MethodInstance for Main.O.naivesum(::Main.O.Bar{Vector{Float64}})
  from naivesum(x::Main.O.AbstractFoo) in Main.O at In[1]:7
Arguments
  #self#::Core.Const(Main.O.naivesum)
  x::Main.O.Bar{Vector{Float64}}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  s::Main.O.Bar{Float64}
  i::Int64
Body::Main.O.Bar{Float64}
1 ─       (s = Main.O.elzero(x))
│   %2  = Main.O.length(x)::Int64
│   %3  = (1:%2)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Main.O.Bar{Float64}
│   %12 = Base.getindex(x, i)::Main.O.Bar{Float64}
│         (s = %11 + %12)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s



In [5]:
@time O.naivesum(foo)
@time O.naivesum(foo)
@time O.naivesum(foo)

  0.092900 seconds (3.01 M allocations: 46.592 MiB, 6.87% gc time, 27.41% compilation time)
  0.064759 seconds (3.00 M allocations: 45.769 MiB, 10.00% gc time)
  0.065813 seconds (3.00 M allocations: 45.769 MiB, 9.28% gc time)


Main.O.Foo(289.7356483316737)

In [6]:
@time O.naivesum(bar)
@time O.naivesum(bar)
@time O.naivesum(bar)

  0.007645 seconds (15.18 k allocations: 873.484 KiB, 86.62% compilation time)
  0.001031 seconds (1 allocation: 16 bytes)
  0.001043 seconds (1 allocation: 16 bytes)


Main.O.Bar{Float64}(289.7356483316737)